In [1]:
%%capture
!pip install tiktoken

In [2]:
from importlib.metadata import version

import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

In [3]:
with open("./the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


- Ta sẽ xóa 50 tokens đầu tiên

In [4]:
enc_sample = enc_text[50:]

- Sinh cặp `input - target`

In [5]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1 : context_size + 1]
print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [6]:
for i in range(1, context_size + 1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "--->", desired)

[290] ---> 4920
[290, 4920] ---> 2241
[290, 4920, 2241] ---> 287
[290, 4920, 2241, 287] ---> 257


- Để dễ hình dung ta sẽ biểu diễn dạng `text` thay vì `token ID` như ở trên

In [7]:
for i in range(1, context_size + 1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "--->", tokenizer.decode([desired]))

 and --->  established
 and established --->  himself
 and established himself --->  in
 and established himself in --->  a


- Để triển khai `Data Loader` hiệu quả, ta sẽ sử dụng `Dataset` & `DataLoader` của Pytorch.

In [8]:
%%capture
!pip install torch

In [9]:
import torch

torch.__version__

'2.8.0+cpu'

In [10]:
import torch
from torch.utils.data import Dataset, DataLoader    

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        # max_length: length of each row
        self.tokenizer = tokenizer
        self.input_ids = []
        self.target_ids = []
        
        token_ids = tokenizer.encode(txt)

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i : i + max_length]
            target_chunk = token_ids[i + 1 : i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]


In [11]:
def create_dataloader_v1(txt, max_length=256, stride=128, batch_size=4, shuffle=True, drop_last=True):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last)

    return dataloader

In [12]:
dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)    # iter = iterable = có thể lặp
first_batch = next(data_iter)   # next = lấy phần tử tiếp theo
second_batch = next(data_iter)
print(first_batch)
print(second_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]
[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


- Lưu ý rằng `batch_size=4` khá nhỏ và được chọn để minh họa. Trong thực tế, việc training LLM thường dùng `batch_size` ít nhất là `256`.

In [13]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


- Ở trên ta tăng `stride` lên `4` (window sliding mỗi 4 phần tử / lần) mục đích để sử dụng toàn bộ tập dữ liệu đồng thời tránh việc các batch bị chồng lên nhau, vì chồng lấn quá nhiều có thể dẫn đến `overfitting` cao hơn.